<a href="https://colab.research.google.com/github/SocSysSci/B6_NaiveBayesClassifier/blob/main/B6_NaiveBayesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 単純ベイズ分類器

### パッケージの読み込み

ここではガウス分布の単純ベイズ分類器（GaussianNB）を利用。単純ベイズ分類器を含む機械学習パッケージ（scikit-learn）については資料を参照のこと。

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

### 学習セットの用意

学習セットの説明については資料を参照のこと。

In [ ]:
X = np.array([[1, 0, 1, 1, 2, 1, 1, 0, 0],
              [1, 2, 0, 1, 0, 0, 3, 3, 1],
              [2, 0, 1, 0, 3, 1, 0, 2, 1]])
y = np.array([1, 2, 3])

### 単純ベイズ分類器の学習

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(X, y)

In [ ]:
clf.score(X, y)

### 学習済の分類器を用いた推定

In [ ]:
t = np.array([[1, 0, 1, 1, 0, 0, 1, 0, 0]])
clf.predict(t)

# 2. テキストの分類

### パッケージの読み込み

CountVectorizerは単語の数を数えるパッケージで，これを利用してBoWベクトルを作成します。

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

### 学習セットの用意

In [ ]:
s = ['今日 は とても 天気 が いい',
     '今日 は 晴れ です',
     '天気 が いい 日 は 晴れ です']

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(s)
X = vectorizer.transform(s).toarray()

In [ ]:
y = np.array([1, 2, 3])

### 単純ベイズ分類器の学習

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(X, y)

In [ ]:
clf.score(X, y)

### 学習済分類器を用いた推定

In [ ]:
t = vectorizer.transform(["明日 は 天気 が いい"]).toarray()
clf.predict(t)

# 3. 長いテキストの分類

### janome（形態素解析）のインストール

In [ ]:
!pip install janome

### パッケージの読み込み

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from janome.tokenizer import Tokenizer

### 学習用文書の読み込み
まず，テキストファイルをアップロードします。
続いて以下のセルを実行し，15つアップしたうち，12を学習用に，3つはテスト用にします。

In [ ]:
# テキスト読み込み用の関数定義
def file_load(file_name):
  text = None
  with open(file_name, "r") as f:
    text = "\n".join(f.readlines())
  return text

In [ ]:
file_list = [
    # 夏目漱石
    "bot_content.txt",
    "34ro_content.txt",
    "kokoro_content.txt",
    "sore_content.txt",
    "meian_content.txt",
    # 芥川竜之介
    "jigokuhen_content.txt",
    "kappa_content.txt",
    "syuju_content.txt",
    # 太宰治
    "ningen_content.txt",
    "shayou_content.txt",
    "hamlet_content.txt",
    "pandora_content.txt"
]
text_list = []
for file in file_list:
  text_list.append(file_load(file))

In [ ]:
# 最初のテキスト（坊ちゃん）の最初の1000文字を表示
text_list[0][0:1000]

### クレンジング

まずクレンジング用の関数を定義して，そのあとクレンジング

In [ ]:
import re

def cleansing(text):
  clean_text = re.sub("\s", "", text)                     # 余分な空白（改行や字下げの空白）を除去
  clean_text = clean_text.replace("<br/>", "\n")          # <br/>タグを改行に変換
  clean_text = re.sub(r"<rp>[^<]+</rp>", "", clean_text)  # ルビの前後の括弧を除去
  clean_text = re.sub(r"<rt>[^<]+</rt>", "", clean_text)  # ルビのテキストを除去
  clean_text = re.sub(r"<[^>]+>", "", clean_text)         # それ以外のタグを除去
  return clean_text

In [ ]:
clean_text_list = []
for text in text_list:
  clean_text_list.append(cleansing(text))

In [ ]:
# 最初のテキストの最初の1000文字を表示
clean_text_list[0][0:1000]

### 分かち書きにする

In [ ]:
# 分かち書き用の関数定義
def wakati_text(text, pos=["名詞", "動詞"]):
    tokenizer = Tokenizer()
    doc = tokenizer.tokenize(text)
    wakati = None
    word_list = []
    for token in doc:
        p = token.part_of_speech.split(",")[0]
        if p in pos:
            word_list.append(token.base_form)
    if 0 < len(word_list):
        wakati = " ".join(word_list)
    return wakati

12個のテキストを分かち書きするのにすごく時間がかかるので，最初の1000文字だけ使います。

In [ ]:
wakati_list = []
for clean_text in clean_text_list:
    wakati_list.append(wakati_text(clean_text[0:1000], ["名詞", "動詞"]))

In [ ]:
# 最初のテキストデータの最初の1000文字を表示
wakati_list[0][0:1000]

### BoWの計算

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(wakati_list)
X = vectorizer.transform(wakati_list).toarray()

### テキストごとのクラスを用意
1：夏目漱石 2：芥川竜之介　3:太宰治

In [ ]:
y = np.array([1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3])

### 単純ベイズ分類器の学習

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(X, y)

In [ ]:
clf.score(X, y)

### テスト用文書の読み込み

In [ ]:
test_file_list = ["waganeko_content.txt", "kumo_content.txt", "melos_content.txt"]
test_text_list = []
for file in test_file_list:
    test_text_list.append(file_load(file))

### クレンジング

In [ ]:
test_clean_list = []
for text in test_text_list:
  test_clean_list.append(cleansing(text))

### テスト用文書の分かち書き

最初の1000文字だけ使っています。

In [ ]:
test_wakati_list = []
for clean_text in test_clean_list:
  test_wakati_list.append(wakati_text(clean_text[0:1000], ["名詞", "動詞"]))

In [ ]:
test_wakati_list[0][0:1000]

### BoWの計算

In [ ]:
T = vectorizer.transform(test_wakati_list).toarray()

### 学習済の分類器を用いて推定

In [ ]:
clf.predict(T)